In [60]:
#import modules here
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine
import time

In [61]:
#define user agent for my fantasy API use
agent = "brockr1"
#create db name - this will be SQLite
db_name = "test"

In [62]:
# create sqlite engine for a db using name defined above
engine = create_engine(f"sqlite:///{db_name}.db", echo=False)
#create connection to the engine
conn = engine.connect()

# Weekly Results
* This section covers functions from Weekly Results API requests

In [68]:
#define function for cleaning players table
def df_players_cleaner(df_players):

    df_players = df_players.astype({
        "id": int,
        "shouldStart": int,
        "score": float
    })

    df_players = df_players.rename(columns = {"shouldStart": "should_start"})

    return df_players

In [64]:
#define function for cleaning df_teams
def df_team_cleaner(df_teams):
    
    df_teams.drop(columns = ["player","comments"],inplace=True)
    
    df_teams = df_teams.astype({
        "score": float,
        "isHome": int,
        "id": int,
    })

    df_teams = df_teams.rename(columns = {"isHome": "is_home"})

    return df_teams

In [65]:
#define function for creating matchups table - !must run df_team_cleaner first!
def matchup_table_generator(df_teams):
    
    total_teams = len(df_teams)
    team1 = list(range(0,total_teams-1,2))
    team2 = list(range(1,total_teams,2))

    df_matchups = pd.DataFrame({
        "team1_id":list(df_teams.iloc[team1]["id"]),
        "team1_score":list(df_teams.iloc[team1]["score"]),
        "team2_id":list(df_teams.iloc[team2]["id"]),
        "team2_score":list(df_teams.iloc[team2]["score"])
    })

    df_matchups['winning_team'] = np.select([df_matchups['team1_score'] > df_matchups['team2_score'], df_matchups['team1_score'] < df_matchups['team2_score']], [df_matchups["team1_id"],df_matchups["team2_id"]], default="error")

    return df_matchups

In [91]:
#pulls weekly score data and saves to SQL server
#pulling more than 3 years at a time will get you locked out of the myfantasy API. it is best to pull in batches.
def weekly_score_pull(years,agent=agent,conn=conn):

    i = 1
    for year in years:
        if i>1:
            print("pausing to prevent lockout of API")
            time.sleep(10)
        
        print("begin parsing year:",year)
        request_url = f"https://www54.myfantasyleague.com/{year}/export?L=20896&W=YTD&TYPE=weeklyResults&JSON=1&{agent}"
        req = requests.get(request_url)
        print("year",year,"connection results:",req)
        req_data = req.json()
            
        weekly_results = req_data['allWeeklyResults']['weeklyResults']

        for single_week in weekly_results:
            if "matchup" not in single_week:
                break
            df_players = pd.json_normalize(single_week["matchup"],record_path=["franchise","player"])
            df_teams = pd.json_normalize(single_week["matchup"],record_path=["franchise"])
            
            #cleaning tables
            df_players = df_players_cleaner(df_players)
            df_teams = df_team_cleaner(df_teams)
            df_matchups = matchup_table_generator(df_teams)

            #adding weeks, years, regular_season to data_tables
            df_players["week"] = int(single_week["week"])
            df_players["year"] = year
            df_players["regular_season"] = int(single_week["matchup"][0]["regularSeason"])
            
            df_teams["week"] = int(single_week["week"])
            df_teams["year"] = year
            df_teams["regular_season"] = int(single_week["matchup"][0]["regularSeason"])
            
            df_matchups["week"] = int(single_week["week"])
            df_matchups["year"] = year
            df_matchups["regular_season"] = int(single_week["matchup"][0]["regularSeason"])        

            #writting to SQL
            df_players.to_sql("weekly_players", conn, if_exists="append");
            df_teams.to_sql("weekly_teams", conn, if_exists="append");
            df_matchups.to_sql("weekly_matchup", conn, if_exists="append");
            print("finished year:",year,"week",single_week["week"])
        
        i = i+1
        


In [67]:
weekly_score_pull([2017,2018,2019,2020,2021],agent=agent,conn=conn)

begin parsing year: 2017
year 2017 connection results: <Response [200]>
finished year: 2017 week 1
finished year: 2017 week 2
finished year: 2017 week 3
finished year: 2017 week 4
finished year: 2017 week 5
finished year: 2017 week 6
finished year: 2017 week 7
finished year: 2017 week 8
finished year: 2017 week 9
finished year: 2017 week 10
finished year: 2017 week 11
finished year: 2017 week 12
finished year: 2017 week 13
finished year: 2017 week 14
finished year: 2017 week 15
finished year: 2017 week 16
pausing to prevent lockout of API
begin parsing year: 2018
year 2018 connection results: <Response [200]>
finished year: 2018 week 1
finished year: 2018 week 2
finished year: 2018 week 3
finished year: 2018 week 4
finished year: 2018 week 5
finished year: 2018 week 6
finished year: 2018 week 7
finished year: 2018 week 8
finished year: 2018 week 9
finished year: 2018 week 10
finished year: 2018 week 11
finished year: 2018 week 12
finished year: 2018 week 13
finished year: 2018 week 14


# Players
* This section covers functions from Weekly Results API requests

In [93]:
def clean_simple_players(players):
    players = players.astype({"id":int})
    players.rename(columns = {"status":"rookie"},inplace = True)
    players["rookie"].replace({"R":"yes"},inplace = True)
    players["rookie"].fillna("No", inplace= True)
    return players

In [95]:
def simple_player_pull (years,agent=agent,conn=conn):
    simple_players = pd.DataFrame()
    i = 1
    for year in years:
        if i>1:
            print("pausing to prevent lockout of API")
            time.sleep(10)
        print("begin parsing year:",year)
        request_url = f"https://www54.myfantasyleague.com/{year}/export?L=20896&TYPE=players&JSON=1&{agent}"
        req = requests.get(request_url)
        req_data = req.json()
        print("connection results:",req)
        
        players = pd.json_normalize(req_data["players"],record_path=["player"])
        
        players = clean_simple_players(players)
        players["year"] = year
        
        players.to_sql("simple_players", conn, if_exists="append")
        i =i+1


    

In [96]:
simple_player_pull([2017,2018,2019,2020,2021],agent=agent,conn=conn)

begin parsing year: 2017
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2018
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2019
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2020
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2021
connection results: <Response [200]>


In [99]:
#example SQL query to test scores table

sql = """
SELECT * 
FROM simple_players
WHERE id = 11367;
"""
# Read the sql query into pandas to view the results
pd.read_sql_query(sql, engine)

,index,position,id,name,team,rookie,year
0,1231,RB,11367,"Juszczyk, Kyle",SFO,No,2017
1,1044,RB,11367,"Juszczyk, Kyle",SFO,No,2018
2,881,RB,11367,"Juszczyk, Kyle",SFO,No,2019
3,766,RB,11367,"Juszczyk, Kyle",SFO,No,2020
4,685,RB,11367,"Juszczyk, Kyle",SFO,No,2021
